In [ ]:
import carla
import pygame
import time
import random
import numpy as np
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from carla import VehicleLightState

# Names for the 15 steering classes (left → no turn → right)
CLASS_NAMES = [
    "Left Hardest", "Left Harder", "Left Hard", "Left Medium", "Left Light", "Left Slight", "Left Minimal",
    "No Turning",
    "Right Minimal", "Right Slight", "Right Light", "Right Medium", "Right Hard", "Right Harder", "Right Hardest"
]

# PyTorch model: ResNet18 backbone + small head, with an embedding for turn signals
class SteeringClassifier(nn.Module):
    def __init__(self, num_classes=15):
        super(SteeringClassifier, self).__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()  # drop the final FC to use raw 512-d features
        self.cnn = resnet
        self.turn_embed = nn.Embedding(3, 16)  # turn signals -1/0/1 mapped to 0/1/2
        self.fc = nn.Sequential(
            nn.Linear(512 + 16, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, turn_signal):
        x_img = self.cnn(image)
        x_signal = self.turn_embed(turn_signal)
        x = torch.cat((x_img, x_signal), dim=1)
        return self.fc(x)

# Load the trained model and set it to eval mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SteeringClassifier(num_classes=15)
model.load_state_dict(torch.load("../Models/30.01_quaternary_model_final.pth", map_location=device))
model.eval().to(device)

# Basic image preprocessing to 224x224 with ImageNet normalization
image_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Spin up a Pygame window to visualize the drive
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("CARLA Model-Controlled Driving")

# Connect to CARLA and load Town05
client = carla.Client("localhost", 2000)
client.set_timeout(25.0)
client.load_world("Town05")
world = client.get_world()
blueprint_library = world.get_blueprint_library()

# Spawn a Dodge Charger and keep control manual (no autopilot)
vehicle_bp = blueprint_library.filter("vehicle.dodge.charger_2020")[0]
spawn_point = random.choice(world.get_map().get_spawn_points())
vehicle = world.spawn_actor(vehicle_bp, spawn_point)
vehicle.set_autopilot(False)

# We’ll use the spectator to follow the car from behind
spectator = world.get_spectator()

# Add a front RGB camera to the car
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '448')
camera_bp.set_attribute('image_size_y', '252')
camera_bp.set_attribute('fov', '145')
camera_bp.set_attribute('sensor_tick', '0.1')
cam_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
camera = world.spawn_actor(camera_bp, cam_transform, attach_to=vehicle)

# Shared state: latest frame (as Pygame surface + NumPy) and current label text
camera_image = None
camera_np = None
pred_label = "Loading..."

def process_image(image):
    # Convert raw CARLA bytes to RGB numpy, keep a copy for inference, and a scaled surface for display
    global camera_image, camera_np
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    array = array[:, :, :3][:, :, ::-1]  # BGRA → RGB
    camera_np = array.copy()
    camera_image_raw = pygame.surfarray.make_surface(array.swapaxes(0, 1))
    camera_image = pygame.transform.scale(camera_image_raw, (800, 600))

# Start streaming frames to our callback
camera.listen(process_image)

# Simple timing helper for the main loop
clock = pygame.time.Clock()

print("Model-controlled driving started.")
print("Q/E to turn on left/right signal, R to cancel.")
print("ESC or close window to exit.")

# Map each of the 15 classes to a steering value (-1.0 left to +1.0 right)
steering_map = {
    0: -0.8,   1: -0.6,   2: -0.4,   3: -0.25,   4: -0.20,  5: -0.1,  6: -0.05,
    7:  0.0,
    8:  0.05,  9:  0.1, 10:  0.15, 11:  0.25,  12:  0.4,  13:  0.6,  14:  0.8
}

def respawn_vehicle():
    # Tear down the current actors and bring the car back somewhere new
    global vehicle, camera

    if camera.is_listening:
        camera.stop()
    camera.destroy()
    vehicle.destroy()

    # Drop the car at a new spawn point
    spawn_point = random.choice(world.get_map().get_spawn_points())
    vehicle_new = world.spawn_actor(vehicle_bp, spawn_point)
    vehicle_new.set_autopilot(False)

    # Re-attach the camera and listen again
    camera_new = world.spawn_actor(camera_bp, cam_transform, attach_to=vehicle_new)
    camera_new.listen(process_image)

    vehicle = vehicle_new
    camera = camera_new

# Start with a modest throttle; we’ll nudge it with W/S
throttle = 0.3

# Main control loop: read inputs, run the model, and apply controls
try:
    while True:
        clock.tick(60)
        pygame.event.pump()
        keys = pygame.key.get_pressed()

        # Turn signals via Q/E, cancel with R; otherwise keep current light state
        if keys[pygame.K_q]:
            vehicle.set_light_state(VehicleLightState.LeftBlinker)
            turn_signal = -1
        elif keys[pygame.K_e]:
            vehicle.set_light_state(VehicleLightState.RightBlinker)
            turn_signal = 1
        elif keys[pygame.K_r]:
            vehicle.set_light_state(VehicleLightState.NONE)
            turn_signal = 0
        else:
            light_state = vehicle.get_light_state()
            if light_state == VehicleLightState.LeftBlinker:
                turn_signal = -1
            elif light_state == VehicleLightState.RightBlinker:
                turn_signal = 1
            else:
                turn_signal = 0

        # Manual throttle tweaks: W to speed up, S to ease off (with a floor)
        if keys[pygame.K_w]:
            throttle += 0.005
        if keys[pygame.K_s]:
            if throttle > 0.1:
                throttle -= 0.005

        # Hit P to respawn the car and reset throttle
        if keys[pygame.K_p]:
            print("Respawning vehicle...")
            respawn_vehicle()
            throttle = 0.3
            time.sleep(1)  # small debounce so holding P doesn’t spam respawns
            continue

        # Build control each frame and apply throttle
        control = carla.VehicleControl()
        control.throttle = throttle

        # Basic smoothing/hold logic for steering updates
        prev_steering_value = 0.0
        last_steering_time = time.time()
        STEERING_HOLD_S = 0.05  # 50 ms

        if camera_np is not None:
            with torch.no_grad():
                img = image_transform(camera_np).unsqueeze(0).to(device)
                signal = torch.tensor([[turn_signal + 1]], dtype=torch.long).to(device)
                output = model(img, signal.squeeze(1))
                pred_class = output.argmax(dim=1).item()
                pred_label = CLASS_NAMES[pred_class]
                steering_value = steering_map[pred_class]

                # If a blinker is on, don’t allow a near-zero steering suggestion
                if turn_signal == -1 and steering_value > -0.02:
                    steering_value = -0.02
                elif turn_signal == 1 and steering_value < 0.02:
                    steering_value = 0.02

                # Apply steering with a tiny bit of smoothing/holding
                current_time = time.time()

                if steering_value == 0.0 or prev_steering_value == 0.0:
                    # For zeros (new or previous), just set it directly
                    control.steer = steering_value
                    prev_steering_value = steering_value
                    last_steering_time = current_time
                elif steering_value != prev_steering_value:
                    # New nonzero suggestion → average with previous for mild smoothing
                    smoothed_value = (steering_value + prev_steering_value) / 2
                    control.steer = smoothed_value
                    prev_steering_value = steering_value
                    last_steering_time = current_time
                else:
                    # Same nonzero value → hold it for a short window
                    if current_time - last_steering_time >= STEERING_HOLD_S:
                        control.steer = steering_value

        # Apply control to the vehicle
        vehicle.apply_control(control)

        # Keep the spectator camera trailing the car from above/behind
        car_transform = vehicle.get_transform()
        forward_vector = car_transform.get_forward_vector()
        cam_location = car_transform.location - forward_vector * 8 + carla.Location(z=3)
        cam_rotation = carla.Rotation(pitch=-10, yaw=car_transform.rotation.yaw)
        spectator.set_transform(carla.Transform(cam_location, cam_rotation))

        # Draw the camera feed and some HUD text
        if camera_image:
            screen.blit(camera_image, (0, 0))
            font = pygame.font.SysFont(None, 30)
            label_turn = font.render(f"Predicted: {pred_label}", True, (255, 255, 0))
            label_steer = font.render(f"Steering: {steering_value:.3f}", True, (0, 255, 0))
            label_speed = font.render(f"Throttle: {throttle:.1f}", True, (0, 200, 255))

            screen.blit(label_turn, (20, 20))
            screen.blit(label_steer, (20, 50))
            screen.blit(label_speed, (20, 80))

            pygame.display.flip()

        # Quit with the window button or ESC
        for event in pygame.event.get():
            if event.type == pygame.QUIT or keys[pygame.K_ESCAPE]:
                raise KeyboardInterrupt

except KeyboardInterrupt:
    print("Exiting and cleaning up...")

finally:
    camera.stop()
    vehicle.destroy()
    camera.destroy()
    pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\aadav\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aadav\anaconda3\envs\ACS\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: '../Models/30.01_quaternary_model_final.pth'